In [1]:
import numpy as np 
import pandas as pd
from tqdm import tqdm
import os
import json

In [3]:
path = "C:/Users/18565/Desktop/Classes/VA/Project/Code/VA-Project"

In [4]:
df = pd.read_csv(os.path.join(path,"data\Lekagul Sensor Data.csv"))

In [6]:
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

In [9]:
df.set_index('Timestamp',inplace=True)

In [14]:
from datetime import datetime

In [22]:
c = car_df.index[1]-car_df.index[0]

In [25]:
c.total_seconds()

1244.0

In [32]:
cols.index(car_df.iloc[ind]['gate-name'])

24

In [33]:
car_ids = np.unique(df['car-id'])
cols = list(np.unique(df['gate-name']))
values = [-1]*len(cols)
df1 = pd.DataFrame("enter_time_stamp")
df2 = pd.DataFrame()
df3 = pd.DataFrame()
for car_id in car_ids:
    car_df = df[df['car-id']==car_id]
    entry_time = str(car_df.index[0])
    exit_time = str(car_df.index[-1])
    total_time = car_df.index[1]-car_df.index[0]
    total_time = total_time.total_seconds()/60
    values[cols.index(car_df.iloc[0]['gate-name'])] = 0
    for ind in range(1,len(car_df)):
        print(car_df.iloc[ind])
        dif = car_df.index[ind]-car_df.index[ind-1]
        dif = dif.total_seconds()/60
        values[cols.index(car_df.iloc[ind]['gate-name'])] = dif
        break
    break

car-id       20150001010009-284
car-type                      3
gate-name         general-gate1
Name: 2015-07-01 13:20:53, dtype: object


In [29]:
cols

['camping0',
 'camping1',
 'camping2',
 'camping3',
 'camping4',
 'camping5',
 'camping6',
 'camping7',
 'camping8',
 'entrance0',
 'entrance1',
 'entrance2',
 'entrance3',
 'entrance4',
 'gate0',
 'gate1',
 'gate2',
 'gate3',
 'gate4',
 'gate5',
 'gate6',
 'gate7',
 'gate8',
 'general-gate0',
 'general-gate1',
 'general-gate2',
 'general-gate3',
 'general-gate4',
 'general-gate5',
 'general-gate6',
 'general-gate7',
 'ranger-base',
 'ranger-stop0',
 'ranger-stop1',
 'ranger-stop2',
 'ranger-stop3',
 'ranger-stop4',
 'ranger-stop5',
 'ranger-stop6',
 'ranger-stop7']